In [1]:
import os
os.environ['CIMG_CIM_PROFILE'] = 'cim18gmdm'
os.environ['CIMG_VALIDATION_LOG_LEVEL'] = 'INFO'
import cimgraph.data_profile.cim18gmdm as cim
from cimgraph import utils
from mermaid import Mermaid

In [ ]:
# Manual Replace:
# TransformerEndInfo.ratedU -> ConductingAssetInfo.ratedVoltage
# Delete TapChanger.normalStep


In [ ]:
from cimgraph.databases import XMLFile
from cimgraph.models import FeederModel

file = XMLFile(filename='final9500node_CIM100x.XML')

network = FeederModel(container=cim.Feeder(), connection=file)

In [ ]:
limit_types = [cim.OperationalLimit, cim.OperationalLimitSet, cim.CurrentLimit, cim.VoltageLimit, cim.OperationalLimitType]
for limit_type in limit_types:
    for obj in network.list_by_class(limit_type):
        network.delete(obj)
        
for node in network.list_by_class(cim.ConnectivityNode):
    node.OperationalLimitSet = None
    
for terminal in network.list_by_class(cim.Terminal):
    terminal.OperationalLimitSet = None

In [ ]:
for topo in network.list_by_class(cim.TopologicalNode):
    network.delete(topo)
    
for node in network.list_by_class(cim.ConnectivityNode):
    node.TopologicalNode = None
    node.ConnectivityNodeContainer = None
    
for topo in network.list_by_class(cim.TopologicalIsland):
    network.delete(topo)

In [ ]:
Sectionalizers = ['TSW320328_SW','TSW568613_SW','LN0621886_SW','A333_48332_SW','A8645_48332_SW',
                  'WF586_48332_SW','TSW803273_SW','WF856_48332_SW','WG127_48332_SW','LN4625713_SW',
                  'TSW10891089_SW','LN05534967_SW','WD701_48332_SW', 'A8869_48332_SW']

Disconnecters = ['DG1069MT1_sw', 'DG1089DIES_sw', 'DG1089LNG_sw','DG1142LNG_sw','DG1209DIES_sw']

Breakers = ['Line.HVMV115B1_sw', 'HVMV69S1B1_sw', 'HVMV69S1B2_sw', 'HVMV69S1B3_sw', 'HVMV69S1B4_sw',
            'HVMV69S2B1_sw', 'HVMV69S2B2_sw', 'HVMV69S2B3_sw', 'HVMV69S3B1_sw', 'HVMV69S3B2_sw']

In [ ]:
def switch_replacer(switch_name, new_class: cim.Switch):
    old_sw = network.find_by_attribute(cim.LoadBreakSwitch, 'name', switch_name)[0]
    new_sw = new_class(identifier=old_sw.identifier)
     

In [ ]:
utils.write_xml(network = network, filename='9500_no_limits.xml', 
                namespaces={'cim': 'http://iec.ch/TC57/CIM101#',
                            'gmdm': 'http://epri.com/gmdm#',
                            'eu': 'http://iec.ch/TC57/CIM100-European#',
                            'nc': 'http://entsoe.eu/ns/nc#',
                            'gb': 'http://GB/placeholder/ext#',
                            'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'})

In [ ]:
os.environ['CIMG_VALIDATION_LOG_LEVEL'] = 'WARNING'

file = XMLFile(filename='./CIM18GMDM/gmdm_9500_node_v0.xml',
               namespaces={'cim': 'http://iec.ch/TC57/CIM101#',
                            'gmdm': 'http://epri.com/gmdm#',
                            'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'})
network = FeederModel(container=cim.Feeder(), connection=file)


In [ ]:
utils.write_xml(network = network, filename='./CIM18GMDM/gmdm_9500_node_v0.xml', 
                namespaces={'cim': 'http://iec.ch/TC57/CIM101#',
                            'gmdm': 'http://epri.com/gmdm#',
                            'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'})

In [ ]:
inverter = network.next(cim.PowerElectronicsConnection)
inverter.pprint()

In [ ]:
deleted = 0
for location in network.list_by_class(cim.Location):
    is_switch = False
    for psr in location.PowerSystemResources:
        if isinstance(psr, cim.Switch):
            is_switch = True
    if not is_switch:
        psr.Location = None
        for point in location.PositionPoints:
            network.delete(point)
        network.delete(location)
        deleted +=1
print(f'{deleted} Location objects deleted from model')

In [ ]:
utils.write_xml(network, '9500_no_loc.xml')

In [ ]:
# os.environ['CIMG_CIM_PROFILE'] = 'cim17v40'
# os.environ['CIMG_VALIDATION_LOG_LEVEL'] = 'warning'
# import cimgraph.data_profile.cim17v40 as cim

# file = XMLFile(filename='9500_no_limits.xml')
# network = FeederModel(container=cim.Feeder(), connection=file)